<a href="https://colab.research.google.com/github/kimdonggyu2008/music_generation/blob/main/melodypreprocessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/음악_생성

/content/drive/MyDrive/음악_생성


In [ ]:
class MelodyPreprocessor: #학습을 위한 데이터셋 세팅
  # 데이터는 음-길이로 이루어져 있음
  def __init__(self,dataset_path,batch_size=32): #학습 초기세팅, 데이터 파일 위치, 반복 횟수
    self.dataset_path=dataset_path
    self.batch_size=batch_size
    self.tokenizer=Tokenizer(filters="",lower=False,split=",") #특성 토큰화(음, 길이)
    self.max_melody_length=None
    self.number_of_tokens=None


  @property
  def number_of_tokens_with_padding(self): #요소갯수 반환, 1개면 0개로 뜨므로 +1해줌
    return self.number_of_tokens+1

  def create_training_dataset(self):
    dataset=self._load_dataset() #토큰화 된 데이터 가져오기((음,길이)의 리스트)
    parsed_melodies=[self._parse_melody(melody) for melody in dataset] #각 데이터에  대해서 반복
    tokenized_melodies=self._tokenize_and_encode_melodies( #
        parsed_melodies
    )
    self._set_max_melody_length(tokenized_melodies)
    self._set_number_of_tokens()
    input_sequences,target_sequences=self._create_sequence_pairs(tokenized_melodies)
    tf_training_dataset=self._convert_to_tf_dataset(
        input_sequences,target_sequences
    )
    return tf_training_dataset

  def _load_dataset(self):
    with open(self.dataset_path,"r") as f:
      return json.load(f) #json파일 읽어옴

  def _parse_melody(self,melody_str):
    return melody_str.split(",") #멜로디를 ,를 기준으로 분할(리스트화)

  def _tokenize_and_encode_melodies(self,melodies):
    self.tokenizer.fit_on_texts(melodies)
    tokenized_melodies=self.tokenizer.texts_to_sequences(melodies)
    return tokenized_melodies

  def _set_max_melody_length(self,melodies):
    self.max_melody_length=max([len(melody) for melody in melodies]) #최대 멜로디 길이 가져옴(모든 곡 중에서)

  def _set_number_of_tokens(self):
    self.number_of_tokens=len(self.tokenizer.word_index) #각 데이터가 몇개의 요소를 가지는지 확인, 딕셔너리의 인덱스값

  def _create_sequence_pairs(self,melodies):
    input_sequences,target_sequences=[],[]
    for melody in melodies: #각 위치의 멜로디에 대해서 반복
      for i in range(1,len(melody)): # 멜로디 마지막까지 반복 [1,2,3,4], 현재위치 4의 경우
        input_seq=melody[:i] #처음부터 현재 위치 직전까지 [1,2,3]
        target_seq=melody[1:i+1] #두번째부터 현재위치까지 [2,3,4]

        padded_input_seq=self._pad_sequence(input_seq) #
        padded_target_seq=self._pad_sequence(target_seq)
        input_sequences.append(padded_input_seq)
        target_sequences.append(padded_target_seq)
    return np.array(input_sequences), np.array(target_sequences)

  def _pad_sequence(self,sequence):
    return sequence+[0]*(self.max_melody_length-len(sequence)) #길이가 다른 각 곡에 대하여, 가장 긴 곡의 길이에 맞춰서 [0]을 추가함

  def _convert_to_tf_dataset(self,input_sequences,target_sequences):
    dataset=tf.data.Dataset.from_tensor_slices(
        (input_sequences,target_sequences)
    )
    shuffled_dataset=dataset.shuffle(buffer_size=1000)
    batched_dataset=shuffled_dataset.batch(self.batch_size)
    return batched_dataset

if __name__=="__main__":
  preprocessor=MelodyPreprocessor("dataset.json",batch_size=32)
  training_dataset=preprocessor.create_training_dataset()